# Pep Free 

In [ ]:
import sys
sys.path.append("../..")
import numpy as np
from run_pipeline import run_pipeline
from Model_analysis.outputs import *
import matplotlib.pyplot as plt
import torch

In [ ]:
protein_id = "pep_free"
base_data_dir = "../../data/palantir_data/pep_free"

pdb_file = f"{base_data_dir}/{protein_id}.pdb"
atom_selection = "backbone"

latent_dim = 16
epochs = 300
batch_size = 64
kl_beta = 1e-3
seed = 42
device = "cuda"

replicas = [0, 1, 2]


experiments = [
    {
        "name": "shuffle_allframes",
        "use_all_frames": True,
        "shuffle_learn": True,
        "base_out_dir": "../../Model_results/shuffle_allframes/pep_free"
    },
    {
        "name": "shuffle_not_allframes",
        "use_all_frames": False,
        "shuffle_learn": True,
        "base_out_dir": "../../Model_results/shuffle_not_allframes/pep_free"
    },
    {
        "name": "not_shuffle_allframes",
        "use_all_frames": True,
        "shuffle_learn": False,
        "base_out_dir": "../../Model_results/not_shuffle_allframes/pep_free"
    },
    {
        "name": "not_shuffle_not_allframes",
        "use_all_frames": False,
        "shuffle_learn": False,
        "base_out_dir": "../../Model_results/not_shuffle_not_allframes/pep_free"
    }
]


for exp in experiments:

    print(f"\n**************************************")
    print(f"File Name: {exp['name']}")
    print(f"Parameters:\n use_all_frames={exp['use_all_frames']} | shuffle={exp['shuffle_learn']}")
    print(f"**************************************")

    for rep in replicas:
        xtc_file = f"{base_data_dir}/{protein_id}_rep_{rep}.xtc"
        out_dir = f"{exp['base_out_dir']}/{protein_id}_rep_{rep}"

        print(f"\n--- RUNNING replica {rep} ---")

        run_pipeline(
            xtc_file=xtc_file,
            pdb_file=pdb_file,
            protein_id=protein_id,
            rep_num=str(rep),
            atom_selection=atom_selection,
            out_dir=out_dir,
            latent_dim=latent_dim,
            epochs=epochs,
            batch_size=batch_size,
            kl_beta=kl_beta,
            use_all_frames=exp["use_all_frames"],
            shuffle_learn=exp["shuffle_learn"],
            seed=seed,
            device=device
        )


In [ ]:
base_dir = "../../Model_results"

conditions = [
    "shuffle_allframes",
    "shuffle_not_allframes",
    "not_shuffle_allframes",
    "not_shuffle_not_allframes"
]

protein_id = "pep_free"
reps = [0, 1, 2]

for condition in conditions:
    out_dirs = []

    for rep in reps:
        out_dir = (
            f"{base_dir}/{condition}/{protein_id}/{protein_id}_rep_{rep}"
        )
        out_dirs.append(out_dir)

    experiment_dirs = model_accuracy_results(
        out_dir=out_dirs[0],
        out_dir_1=out_dirs[1],
        out_dir_2=out_dirs[2]
    )

    model_performance_results(experiment_dirs)